# Lab 1 - drzewa decyzyjne
*"pro" version*

Autorstwa:
- Wojciech Kot  151879
- Julia Samp    151775
z grupy L7, zajęcia piątkowe 11:45

--- workspace - usunąć przed wysłaniem ---
• entropy calculation,
• conditional entropy calculation,
• information gain calculation,
• gain ratio calculation,
• simple drawing of a decision tree (there can be really primitive way of
drawing or usage of any ready package to do only this drawing) with the
information how many examples of each class are in each leaf.


- biblioteka treelib może być przydatna

In [1]:
# kod
